In [75]:
import pandas as pd
import numpy as np

In [76]:
path_name = 'C:/Users/Usuario/Downloads/DenunciesGUB-2023_GD_parcial.csv'

In [77]:
delimiter = ';'

In [52]:
# How many column headers do we have?
max_columns = max(open(path_name, 'r'), key = lambda x: x.count(delimiter)).count(delimiter)
max_columns

22

In [59]:
df_raw = pd.read_csv(path_name, 
                     encoding='latin-1', 
                     keep_default_na=True,
                     header = None,
                     nrows=1,
                     delimiter=';'
                    )

columns = df_raw.values

In [60]:
columns

array([['Any Data Infraccio', 'Expedient', 'Codi Infraccio',
        'Codi infracció', 'Ordenança', 'Descripció ordenança',
        'Precepte/Article infringit', 'Carrer Infracció',
        'Num carrer infracció', 'Barri Infracció', 'Nom barri',
        'Districte Infracció', 'Tipologia radar', 'Data Infraccio',
        'Hora Infraccio', 'Tipus de vehícle', 'Descripció tipus vehicle',
        'Marca del vehicle', 'Model del vehícle', 'Velocitat circulació',
        'Velocitat límit', 'Total Import', 'Imp cobrat']], dtype=object)

In [69]:
df_raw[23].isna().sum(),len(df_raw)

(356659, 356659)

The last column does not contain any values. So we can import again with less columns

In [78]:
df_raw = pd.read_csv(path_name, 
                     encoding='latin-1', 
                     #keep_default_na=True,
                     #header = None,
                     #names = list(range(0,max_columns)),
                     #skiprows = 1,
                     delimiter=delimiter
                    )

C:\Users\Usuario\AppData\Local\Temp\ipykernel_2712\251316281.py:1: DtypeWarning: Columns (11,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(path_name,


In [81]:
# How many rows?
len(df_raw)

356659

In [84]:
for col in df_raw.columns:
    s = df_raw[col]
    vc = s.value_counts()
    if len(vc)<len(df_raw)/100:
        print(f'Column: {col}')
        print('___')
        print(vc)
        print('___')
    else:
        print(f'Column {col} has too many unique values.')

Column: Any Data Infraccio
___
2023    356659
Name: Any Data Infraccio, dtype: int64
___
Column Expedient has too many unique values.
Column: Codi Infraccio
___
1211    103577
1071     65291
1660     32892
1210     18260
1072     16184
         ...  
1089         1
1650         1
1536         1
1593         1
1408         1
Name: Codi Infraccio, Length: 440, dtype: int64
___
Column: Codi infracció
___
SUPERAR LIMIT VELOCITAT SENYAL ESPECIFIC                        104439
MANCA DE COMPROVANT HORARI                                       65291
NO RESPECTAR RESTRICCIONS ZONES BAIXES EMISSIONS                 32892
CIRCULA AMB EXCES DE VELOCITAT PER RAO DE VIA                    18260
SUPERAR LIMIT HORARI                                             16184
                                                                 ...  
PORTAR MECANISME DETECCIO RADAR CINEMOMETRE                          1
REINTEGRAR-SE AL CARRIL SENSE SENYALITZ                              1
PARAR SOBRE CARRILS TRAMVIA

In [85]:
df_raw.columns

Index(['Any Data Infraccio', 'Expedient', 'Codi Infraccio', 'Codi infracció',
       'Ordenança', 'Descripció ordenança', 'Precepte/Article infringit',
       'Carrer Infracció', 'Num carrer infracció', 'Barri Infracció',
       'Nom barri', 'Districte Infracció', 'Tipologia radar', 'Data Infraccio',
       'Hora Infraccio', 'Tipus de vehícle', 'Descripció tipus vehicle',
       'Marca del vehicle', 'Model del vehícle', 'Velocitat circulació',
       'Velocitat límit', 'Total Import', 'Imp cobrat'],
      dtype='object')

In [87]:
df_raw.head(2)[['Nom barri', 'Districte Infracció', 'Tipologia radar']]

,Nom barri,Districte Infracció,Tipologia radar
0,el Poblenou,10,
1,el Camp de l'Arpa del Clot,10,


In [88]:
cols_to_keep = ['Codi infracció','Descripció ordenança','Carrer Infracció','Nom barri','Data Infraccio',
                'Hora Infraccio','Descripció tipus vehicle','Marca del vehicle','Model del vehícle',
                'Velocitat circulació','Velocitat límit','Total Import','Imp cobrat'
               ]

In [94]:
df = df_raw[cols_to_keep].copy()

In [95]:
df['Data Infraccio'] = pd.to_datetime(df['Data Infraccio'], format='%Y%m%d')

In [98]:
df['Hora Infraccio'] = df['Hora Infraccio'].apply(lambda x: x[0:2])

In [102]:
df['Imp cobrat'].value_counts()

0         107588
0          71783
50         44126
25         22565
50         19878
           ...  
241,11         1
31,5           1
108,61         1
502            1
75             1
Name: Imp cobrat, Length: 94, dtype: int64

In [107]:
df['Imp cobrat'] = df['Imp cobrat'].apply(lambda x: float(str(x).replace(',','.')))

In [108]:
df.to_parquet('./denuncies_partial_2023.parquet')